In [ ]:
import cv2
import numpy as np
from skimage.filters import frangi, gabor
from skimage import measure, morphology

def master_control(image):
    # 이미지의 B, G, R 채널 분리 (여기서는 녹색 채널 사용)
    b, g, r = cv2.split(image)
    
    # 녹색 채널에 Frangi 필터 적용 (주름의 선형 구조 강조)
    sk_frangi_img = frangi(g, scale_range=(0, 1), scale_step=0.01, beta=1.5, gamma=0.01)
    sk_frangi_img = morphology.closing(sk_frangi_img, morphology.disk(1))
    
    # 녹색 채널에 Gabor 필터 적용 (여러 방향의 주름 강조)
    sk_gabor_img_1, _ = gabor(g, frequency=0.35, theta=0)
    sk_gabor_img_2, _ = gabor(g, frequency=0.35, theta=np.deg2rad(45))
    sk_gabor_img_3, _ = gabor(g, frequency=0.35, theta=np.deg2rad(90))
    sk_gabor_img_4, _ = gabor(g, frequency=0.35, theta=np.deg2rad(360))  # 360°는 0°와 동일
    sk_gabor_img_1 = morphology.opening(sk_gabor_img_1, morphology.disk(2))
    sk_gabor_img_2 = morphology.opening(sk_gabor_img_2, morphology.disk(1))
    sk_gabor_img_3 = morphology.opening(sk_gabor_img_3, morphology.disk(2))
    sk_gabor_img_4 = morphology.opening(sk_gabor_img_4, morphology.disk(2))
    
    # Frangi와 Gabor 결과 결합: Gabor(theta=45)에 0.1, Frangi에 0.9의 가중치 부여
    all_img = cv2.add(0.1 * sk_gabor_img_2.astype(np.float32), 0.9 * sk_frangi_img.astype(np.float32))
    all_img = morphology.closing(all_img, morphology.disk(1))
    
    # Thresholding을 통해 이진화 (임계값 0.3 사용)
    _, all_img = cv2.threshold(all_img, 0.3, 1, cv2.THRESH_BINARY)
    
    # 연결 영역(연결분할) 라벨링 및 노이즈 제거
    bool_img = all_img.astype(bool)
    label_image = measure.label(bool_img)
    count = 0
    for region in measure.regionprops(label_image):
        if region.area < 10:
            for coord in region.coords:
                all_img[coord[0], coord[1]] = 0
            continue
        if region.eccentricity > 0.98:
            count += 1
        else:
            for coord in region.coords:
                all_img[coord[0], coord[1]] = 0
    
    # 남은 영역에서 중앙축(스켈레톤) 추출
    skel, distance = morphology.medial_axis(all_img.astype(int), return_distance=True)
    skels = morphology.closing(skel, morphology.disk(1))
    return skels, count

def face_wrinkle(path):
    # 이미지 읽기 (cv2.imread는 BGR 포맷으로 읽습니다.)
    image = cv2.imread(path)
    if image is None:
        print("이미지를 찾을 수 없습니다.")
        return
    
    # 주름 스켈레톤과 영역 개수 추출
    skels, count = master_control(image)
    print("검출된 주름 영역 수:", count)
    
    # 원본 이미지 위에 주름 스켈레톤 오버레이 (cv2.imshow로 표시)
    overlay = image.copy()
    # 스켈레톤은 boolean이므로, 255로 스케일링하여 오버레이 (BGR: 빨간색 [0, 0, 255])
    overlay[skels > 0] = [0, 0, 255]
    
    # cv2.imshow로 결과 표시 (Jupyter Notebook이 아닌 로컬 환경에서 사용)
    cv2.imshow("Original Image", image)
    cv2.imshow("Wrinkle Skeleton Mask", (skels * 255).astype(np.uint8))
    cv2.imshow("Overlay (Red: Wrinkle)", overlay)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 실행 예시: 아래 경로를 사용자의 이미지 파일 경로로 수정하세요.
if __name__ == '__main__':
    image_path = "Asian_celebrity_align/shingu/1_shingu_70_m.jpg"
    face_wrinkle(image_path)


/tmp/ipykernel_63298/1022225396.py:11: UserWarning: Use keyword parameter `sigmas` instead of `scale_range` and `scale_range` which will be removed in version 0.17.
  sk_frangi_img = frangi(g, scale_range=(0, 1), scale_step=0.01, beta=1.5, gamma=0.01)


검출된 주름 영역 수: 0


Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation


In [3]:
!pip install scikit-image